In [1]:
################################################# import libraries ###########################################
%load_ext jupyternotify

import pandas as pd
import os
from nltk.corpus import stopwords
import string
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import adjusted_rand_score
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import operator
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import time
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import AgglomerativeClustering
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans
import pickle
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
import pandas as pd


<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import codecs
from tqdm import tqdm

def load_dataset(path):
    dataset = pd.DataFrame(columns=['X', 'y1', 'y2'])
    #print('Loading dataset...')
    with codecs.open(path, "r", encoding='utf-8', errors='ignore') as fdata:
        for line in tqdm(fdata.readlines()):
            line_split = line.split()
            formated = ' '.join(line_split[:-2])
            dataset.loc[-1] = [formated, line_split[-2], line_split[-1]]  # adding a row
            dataset.index = dataset.index + 1  # shifting index
            dataset = dataset.sort_index()  # sorting by index
    return dataset

In [3]:
def load_dataset_ext(path):
    df = pd.read_csv(path, index_col=0)
    df.drop(columns=['count'])
    df_droped = df.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'])
    df_droped.columns = ['y', 'X']
    return df_droped.replace([0, 1], 'OFFENSE').replace(2, 'OTHER')

In [4]:
import pandas as pd

def load_dataset_hate(path):
    dataset = pd.DataFrame(columns=['X', 'y1', 'y2', 'rating'])
    with codecs.open(path, "r", encoding='utf-8', errors='ignore') as fdata:
        for line in tqdm(fdata.readlines()):
            line = line.strip()
            line_split = line.split(';')
            formated = ' '.join(line_split[:-3])
            dataset.loc[-1] = [formated, line_split[-3], line_split[-2], line_split[-1]]  # adding a row
            dataset.index = dataset.index + 1  # shifting index
            dataset = dataset.sort_index()  # sorting by index
    return dataset.drop(dataset.index[[-1]])

In [5]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

stopwords_german = set(stopwords.words('german'))

usernamePattern = re.compile('@[A-Za-z0-9_]{1,15}')
urlPattern = re.compile('(https?:\/\/)[\/.:\w(1-9)]*\s?')
lbrPattern = re.compile('|LBR|')
stemmer = SnowballStemmer("german")
tkz = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

def tokenize(text):
    text = re.sub(usernamePattern, "",  text)
    text = re.sub(urlPattern, "",  text)
    text = re.sub(lbrPattern, "",  text)
    output = []
    tokens = tkz.tokenize(text)
    for token in tokens:
        if token not in stopwords_german:
            if len(token) > 1:
                if token[0] == '#':
                    token = token[1:]
                output.append(token)
    return output

In [6]:
def tokenize_tfidf(text):
    text = re.sub(usernamePattern, "",  text)
    text = re.sub(urlPattern, "",  text)
    text = re.sub(lbrPattern, "",  text)
    output = []
    tokens = tkz.tokenize(text)
    for token in tokens:
        if token not in stopwords_german:
            if len(token) > 1:
                if token[0] == '#':
                    token = token[1:]
                output.append(stemmer.stem(token))
    return output

In [7]:
def load_dataset_ext(path):
    df = pd.read_csv(path, index_col=0)
    df.drop(columns=['count'])
    df_droped = df.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'])
    df_droped.columns = ['y1', 'X']
    return df_droped.replace([0, 1], 'OFFENSE').replace(2, 'OTHER')

In [8]:
dataset = load_dataset(path = '/home/text_mining_project/text_mining_project_2018/evaluation/germeval2018.training.txt')

100%|██████████| 5009/5009 [00:15<00:00, 333.38it/s]


In [9]:
dataset_own = load_dataset(path = '/home/text_mining_project/export.tsv')

100%|██████████| 3189/3189 [00:09<00:00, 350.55it/s]


In [10]:
dataset_david = load_dataset_ext('/home/text_mining_project/t-davidson.csv')

In [11]:
dataset_hate = load_dataset_hate('/home/text_mining_project/text_mining_project_2018/evaluation/german_hatespeech_refugees.csv')

100%|██████████| 470/470 [00:01<00:00, 349.75it/s]


In [12]:
# dataset_full =  pd.concat([dataset, dataset_own])
# dataset_full = dataset_david
dataset_full = dataset_hate

In [13]:
def encode_label_bin(y, predicted_label):
    choose = lambda l : True if l == predicted_label else False
    return [choose(l) for l in y]

y1 = np.array(encode_label_bin(dataset_hate['y1'], 'YES'))
y2 = np.array(encode_label_bin(dataset_hate['y2'], 'YES'))

y = (y1 | y2)*1


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


X = dataset_full['X'].values
# y = dataset_full['y1'].values

# lb = LabelEncoder()
# lb.fit(['OFFENSE','OTHER'])
# y = np.ones(len(y)) - lb.transform(y)

X_train_dataset, X_test_dataset, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features = None
stopwords_german = set(stopwords.words('german'))
# stopwords_german = set(stopwords.words('english'))
tfidf = TfidfVectorizer(tokenizer=tokenize_tfidf, stop_words=stopwords_german, max_features=max_features, ngram_range=(1,3))
print('Transforming documents...')
X_train_tfidf = tfidf.fit_transform(X_train_dataset)
X_test_tfidf = tfidf.transform(X_test_dataset)
print('Transformation finished!')

Transforming documents...
Transformation finished!


In [16]:
model = Word2Vec.load("word2vec_nostem_stopword.model")
# model = Word2Vec.load("word2vec_nostem_stopword_english.model")

In [17]:
from numpy import zeros

def make_bag_of_centroids(X, word_centroid_map, cluster_size):
    centroids_list = []
    for sentence in X:
        centroids = zeros(cluster_size, dtype="float32")
        for word in sentence:
            if word in word_centroid_map:
                centroids[word_centroid_map[word]] += 1
        centroids_list.append(centroids)
    return centroids_list

as_centroid = lambda s: make_bag_of_centroids(s, word2centroid, kmeans_args['n_clusters'])

In [18]:
kmeans_args = {
    'n_clusters': 1000,
}

# clustering = KMeans(**kmeans_args).fit_predict(model.wv.vectors)

clustering = MiniBatchKMeans(**kmeans_args).fit_predict(model.wv.vectors)
pickle.dump(clustering, open('./minibatchkmeans.bin', 'wb'))

/home/koogleschreiber/.local/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning:

init_size=300 should be larger than k=1000. Setting it to 3*k



In [19]:
word2centroid = {k: v for k, v in zip(model.wv.index2word, clustering)}

In [20]:
def merge_vector_lists(x1, x2):
    result = []
    for i in range(len(x1)):
        result.append(x1[i] + x2[i])
    return result

In [21]:
stopwords_german = set(stopwords.words('german'))
# stopwords_german = set(stopwords.words('english'))

def preprocess(X):
    return [tokenize(sentence) for sentence in X]

X_preprocess_train = preprocess(X_train_dataset)
X_preprocess_test = preprocess(X_test_dataset)

X_train_centroid = scale(as_centroid(X_preprocess_train))
X_test_centroid = scale(as_centroid(X_preprocess_test))

from scipy.sparse import csr_matrix, hstack

X_train_centroid_sparse = csr_matrix(X_train_centroid)
X_test_centroid_sparse = csr_matrix(X_test_centroid)


X_train = hstack([X_train_centroid_sparse, X_train_tfidf])
X_test = hstack([X_test_centroid_sparse, X_test_tfidf])


In [22]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)
X_test = model.transform(X_test)

In [23]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# 
# scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
# X_train = scaler.fit_transform(X_train.todense())
# X_test = scaler.transform(X_test.todense())
# 
# ch2 = SelectKBest(chi2, k=2000)
# X_train = ch2.fit_transform(X_train, y_train)
# X_test = ch2.transform(X_test)

In [24]:
import warnings

seed = 42
k = 7
jobs = -1

warnings.filterwarnings("ignore")

In [25]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import cross_val_score


names = ["Linear SVM", "SGDClassifier", "BernoulliNB", "LogisticRegression",
         "KNeighborsClassifier", "AdaBoostClassifier", "Random Forest", "Decision Tree"]

classifiers = [
    LinearSVC(random_state=seed),
    SGDClassifier(max_iter=1000, tol=None),
    BernoulliNB(),
    LogisticRegression(random_state=seed, solver='sag', max_iter=1000),
    KNeighborsClassifier(),
    AdaBoostClassifier(),
    RandomForestClassifier(random_state=seed),
    DecisionTreeClassifier(random_state=seed)
]

print('Searching best estimator...')
print()
best_classifier = None
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, X_train, y_train, cv=k, n_jobs=jobs)
    print('Mean accuracy %s: %0.3f (+/- %0.3f)' % (name, scores.mean(), scores.std() * 2))
    if not best_classifier:
        best_classifier = (name, scores.mean())
    else:
        if best_classifier[1] < scores.mean():
            best_classifier = (name, scores.mean())
print()
print('Best estimator: %s (mean acc %0.3f, %d-fold cross-validation)' % (best_classifier[0], best_classifier[1], k))

Searching best estimator...

Mean accuracy Linear SVM: 0.658 (+/- 0.149)
Mean accuracy SGDClassifier: 0.711 (+/- 0.093)
Mean accuracy BernoulliNB: 0.720 (+/- 0.088)
Mean accuracy LogisticRegression: 0.692 (+/- 0.124)
Mean accuracy KNeighborsClassifier: 0.739 (+/- 0.055)
Mean accuracy AdaBoostClassifier: 0.715 (+/- 0.105)
Mean accuracy Random Forest: 0.742 (+/- 0.069)
Mean accuracy Decision Tree: 0.720 (+/- 0.032)

Best estimator: Random Forest (mean acc 0.742, 7-fold cross-validation)


In [26]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

print('Searching best estimator (F1 score) ...')
print()
best_classifier = None
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, X_train, y_train, cv=k, n_jobs=jobs, scoring='f1')
    print('Mean F1 score %s: %0.3f (+/- %0.3f)' % (name, scores.mean(), scores.std() * 2))
    if not best_classifier:
        best_classifier = (name, scores.mean())
    else:
        if best_classifier[1] < scores.mean():
            best_classifier = (name, scores.mean())
print()
print('Best estimator: %s (mean F1 score %0.3f, %d-fold cross-validation)' % (best_classifier[0], best_classifier[1], k))

Searching best estimator (F1 score) ...

Mean F1 score Linear SVM: 0.542 (+/- 0.200)
Mean F1 score SGDClassifier: 0.396 (+/- 0.230)
Mean F1 score BernoulliNB: 0.492 (+/- 0.222)
Mean F1 score LogisticRegression: 0.636 (+/- 0.139)
Mean F1 score KNeighborsClassifier: 0.458 (+/- 0.146)
Mean F1 score AdaBoostClassifier: 0.527 (+/- 0.196)
Mean F1 score Random Forest: 0.518 (+/- 0.120)
Mean F1 score Decision Tree: 0.553 (+/- 0.093)

Best estimator: LogisticRegression (mean F1 score 0.636, 7-fold cross-validation)


In [27]:
clf = LinearSVC()
clf.fit(X_train, y_train)
predictions=clf.predict(X_test)

In [28]:
from sklearn.metrics import f1_score
f1=f1_score(predictions, y_test)
print("F1-Score predictions: ", f1)

F1-Score predictions:  0.47058823529411764


In [29]:
clf = BernoulliNB(alpha=0.1, binarize=0, class_prior=None, fit_prior=False)
clf.fit(X_train, y_train)
predictions=clf.predict(X_test)

In [30]:
from sklearn.metrics import f1_score
f1=f1_score(predictions, y_test)
print("F1-Score predictions: ", f1)

F1-Score predictions:  0.5882352941176471


In [31]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
predictions=clf.predict(X_test)

In [32]:
from sklearn.metrics import f1_score
f1=f1_score(predictions, y_test)
print("F1-Score predictions: ", f1)

F1-Score predictions:  0.5384615384615385
